In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame
import glob
import requests
# from geoalchemy2 import Geometry, WKTElement

import psycopg2  # (if it is postgres/postgis)
conn = psycopg2.connect(database="postgres", user="wenfeixu", password="",
    host="localhost")
cursor = conn.cursor()


from sqlalchemy import create_engine
engine = create_engine('postgresql://wenfeixu:@localhost:5432/postgres')
    
from geoalchemy2 import Geometry, WKTElement



# Goal of the Notebook

0. Prepare HOLC data (for some duplicate geometries and mis-labeled cities)
1. Read in the cleaned HOLC data
2. Read in relevant Decennial Census data (1930 - 2016, 2016 is 5yr ACS)
3. OPTIONAL: For population-weighted HOLC areas.
4. OPTIONAL: Write the results from 3. to shapefiles, postgis tables, upload to mapbox or carto.

# Datasets Used

- HOLC polygons from University of Richmond Digital Scholarship Lab
- Historical Census Data from IPUMS National Historical GIS database
****

# 0. Prepare HOLC Data
Note: Since 2018 there are new polgons. 

It seems like some  cities are all `map_id`=0. I made the following changes in the Map IDs: 
- Phoenix = 1
- Pueblo, CO = 2
- Rochester = 3
- Davenport, IA = 4
- Covington, KY = 5
- Hartford, CT = 6


In [5]:
### 'holc_polygons_update.shp' is the latest version of the HOLC dataset

holc_all = gpd.read_file('/data/holc_polygons_update/holc_polygons_update.shp').to_crs('epsg:4326')

### remove Buffalo, Oklahoma City, because it's in there twice
holc_all = holc_all[holc_all.map_id.isin([111,235])==False]
holc_all1 = holc_all.groupby(['map_id','HOLC_Grade','SHAPE_Leng']).first().reset_index()
holc_all1.to_file('/data/holc_polygons_update/holc_polygons_update_noZ_dedup.shp')

In [43]:
holc_bronx = gpd.read_file('data/holc_polygons_update/holc_polygons_update_noZ.shp')

In [44]:
holc_bronx1 = holc_bronx[holc_bronx.map_id==98].to_crs('epsg:4326')

In [49]:
holc_all1 = gpd.GeoDataFrame( pd.concat( [holc_all,holc_bronx1], ignore_index=True) )

Convert Map IDs to City name

In [52]:
id_to_city = {27:'New Haven',
             0:'San Antonio',
              1: 'Phoenix',
              2:'Pueblo',
              3:'Rochester',
              4:'Davenport',
              5:'Covington',
              6:'Hartford',
              10:'Birmingham',
              11:'Mobile',
              12:'Montgomery',
              13:'Little Rock',
              15:'Fresno',
              16:'Los Angeles',
              17:'Oakland',
              18:'Sacramento',
              19:'San Diego',
              20:'San Francisco',
              21:'San Jose',
              22:'Stockton',
              23:'Denver',
              27:'New Haven',
              28:'Stamford',
              29:'Jacksonville',
              30:"Miami",
              31:'St. Petersburg',
              32:'Tampa',
              33:'Atlanta',
              34:'Augusta',
              35:'Columbus',
              36:'Macon',
              38:'Council Buffs',
              40:'Des Moines',
              41:'Dubuque',
              42:'Sioux City',
              43:'Waterloo',
              44:'Aurora',
              45:'Chicago',
              46:'Decatur',
              47:'East St. Louis',
              48:"Jolie",
              49:"Rockford",
              50:'Springfield',
              51:'Evansville',
              52:'Fort Wayne',
              53:'Indianapolis',
              54:'Calumet City',
              55:'Muncie',
              56:'South Bend',
              57:"Terre Haute",
              58:'Topeka',
              59:'Wichita',
              61:'Lexington',
              62:'Louisville',
              63:'New Orleans',
              64:'Shreveport',
              68:'Baltimore',
              69:'Battlecreek',
              70:'Bay City',
              71:'Detroit',
              72:'Flint',
              73:'Grand Rapids',
              75:'Kalamazoo',
              76:'Lansing',
              77:'Muskegon',
              78:'Pontiac',
              79:'Sagninaw',
              80:'Duluth',
              81:'Minneapolis',
              83:'St. Paul',
              85:'St. Louis',
              86:'Springfield',
              87:'Saint Joseph',
              90:'Asheville',
              91:'Charlotte',
              92:'Durham',
              93:'Greensboro',
              94:'Winston-Salem',
              95:'Hoboken-Jersey City-East Newark',
              96:'Newark',
              98:'Bronx',
              99:'Brooklyn',
              101:'Manhattan',
              102:'Niagara Falls',
              103:'Poughkeepsie',
              104:'Queens',
              105:'Rochester',
              106:'Schenechtady',
              107:'Staten Island',
              108:'Syracuse',
              109:'Troy',
              110:'Utica',
              111:'Buffalo',
              114:'Akron',
              115:'Cleveland',
              116:'Columbus',
              117:'Dayton',
              118:'Hamilton',
              119:'Lima',
              120:'Lorain',
              121:'Portsmouth',
              122:'Springfield',
              123:'Toledo',
              124:'Warren',
              125:'Youngstown',
              127:'Tulsa',
              128:'Portland',
              132:'Erie',
              134:'Johnstown',
              137:'New Castle',
              138:'Philadelphia',
              139:'Pittsburgh',
              147:'Chattanooga',
              148:'Knoxville',
              154:'Dallas',
              158:'Houston',
              164:'Lynchburg',
              165:'Newport News',
              166:'Norfolk',
              168:'Richmond',
              169:'Roanoke',
              170:'Seattle',
              171:'Spokane',
              172:'Tacoma',
              173:'Kenosha',
              174:'Madison',
              175:'Oshkosh',
              176:'Racine',
              177:'Charleston',
              179:'Wheeling',
              181:'Hartford',
              182:'New Britain',
              183:'Waterbury',
              184:'Brockton',
              185:'Haverhill',
              186:'Holyoke',
              188:'Manchester',
              189:'Atlantic City',
              190:'Hackensack',
              191:'Camden',
              192:'Trenton',
              193:'Albany',
              194:'Binghamton',
              195:'Buffalo',
              196:'Elmira',
              197:'Yonkers',
              198:'Canton',
              201:'Milwaukee',
              224:'Jackson',
              226:'Kansas City',
              236:'Oklahoma City',
              237:'Altoona',
              265:'Arlington',
              266:'Belmont',
              267:'Boston',
              268:'Braintree',
              269:'Brookline',
              270:'Cambridge',
              271:'Chelsea',
              272:'Dedham',
              273:'Everett',
              274:'Lexington',
              275:'Malden',
              276:'Medford',
              277:'Melrose',
              278:'Milton',
              279:'Needham',
              280:'Newton',
              281:'Quincy',
              282:'Revere',
              283:'Saugus',
              284:'Somerville',
              285:'Waltham',
              286:'Watertown',
              287:'Winchester',
              288:'Winthrop',
              319:'Nashville'
             }


In [53]:
holc_all1['map_name'] = holc_all1['map_id'].apply(lambda x:id_to_city[x] )

In [54]:
holc_all1.to_postgis(name="holc_all_dump_4326",con= engine,if_exists='replace')


****

(If re-running, start here)

# 1. Read cleaned HOLC Data

In [3]:
sql = '''
select * from holc_all_dump_4326
'''
holc_all = gpd.GeoDataFrame.from_postgis(sql, conn,crs={'init':'epsg:4326'}, geom_col='geometry' )

/Users/wenfeixu/anaconda3/envs/gds/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [4]:
holc_all.head()

,map_id,HOLC_Grade,SHAPE_Leng,HOLC_ID,HOLC_Lette,polygon_id,SHAPE_Area,sheets,name,OBJECTID,map_name,geometry
0,0,A,17623.230575,None,None,14,5.861855e+06,0,None,0,San Antonio,"POLYGON ((-98.48526 29.45325, -98.49215 29.453..."
1,0,A,21230.295569,None,None,1,9.326094e+06,0,None,0,San Antonio,"POLYGON ((-98.52972 29.46838, -98.52966 29.468..."
2,0,B,3763.973058,None,None,10,7.605122e+05,0,None,0,San Antonio,"POLYGON ((-98.48805 29.41147, -98.48800 29.411..."
3,0,B,5404.821964,None,None,2,1.072495e+06,0,None,0,San Antonio,"POLYGON ((-98.51525 29.45301, -98.51520 29.452..."
4,0,B,6670.183791,None,None,13,1.747458e+06,0,None,0,San Antonio,"POLYGON ((-98.49289 29.48830, -98.49218 29.488..."


# 2. Get All Census Data
From 1930 - 1980 we use census tracts, 1990 onwards are block groups.


Also make sure to prep all the data here. This includes: 
- creating the 'other' (non-white, non-black) 
- adjusting the median income levels for inflation to 2016 levels.
- aggregating categories when necessary

### 1930 
Using census from that year. Has the following columns: 
- Population
- Race by white, black, other

Each of the cities have their own file, so we'll need to join them together.

In [14]:
census_1930_1 = pd.read_csv('data/NHGIS/census_1930/nhgis0059_ds58_1930_tract.csv',index_col=None, header=0)
census_1930_1 = census_1930_1.rename(columns={'BHI001':'population',
                                          'BIQ001':'white_native',
                                          'BIQ002':'white_foreignparents',
                                          'BIQ003':'white_foreign',
                                          'BIQ004':'black',
                                        'BIQ005':'other'})

census_1930_2 = pd.read_csv('data/NHGIS/census_1930/nhgis0059_ds60_1930_tract.csv',index_col=None, header=0)
census_1930_2 = census_1930_2.rename(columns={'BJW001':'population',
                                          'BK1001':'white_native',
                                          'BK1002':'white_foreignparents',
                                          'BK1003':'white_foreign',
                                          'BK1004':'black',
                                          'BK1005':'other' })

census_1930_3 = pd.read_csv('data/NHGIS/census_1930/nhgis0059_ds62_1930_tract.csv',index_col=None, header=0)
census_1930_3 = census_1930_3.rename(columns={'BLO001':'population',
                                          'BLR001':'white_native',
                                          'BLR002':'white_foreignparents',
                                          'BLR003':'white_foreign',
                                          'BLR004':'black',
                                        'BLR005':'other'})

census_1930_4 = pd.read_csv('data/NHGIS/census_1930/nhgis0059_ds63_1930_tract.csv',index_col=None, header=0)
census_1930_4 = census_1930_4.rename(columns={'BLW001':'population',
                                           'BM4001':'white_native',
                                          'BM4002':'white_foreignparents',
                                          'BM4003':'white_foreign',
                                          'BM4004':'black',
                                        'BM4005':'other'})

census_1930_5 = pd.read_csv('data/NHGIS/census_1930/nhgis0059_ds64_1930_tract.csv',index_col=None, header=0)
census_1930_5 = census_1930_5.rename(columns={'BND001':'population'})

census_1930_6 = pd.read_csv('data/NHGIS/census_1930/nhgis0059_ds65_1930_tract.csv',index_col=None, header=0)
census_1930_6 = census_1930_6.rename(columns={'BNE001':'population',
                                          'BNP001':'white_native',
                                          'BNP002':'white_foreignparents',
                                          'BNP003':'white_foreign',
                                          'BNP004':'black',
                                        'BNP005':'other'})

census_1930_7 = pd.read_csv('data/NHGIS/census_1930/nhgis0059_ds66_1930_tract.csv',index_col=None, header=0)
census_1930_7 = census_1930_7.rename(columns={'BOI001':'population'})

census_1930_8 = pd.read_csv('data/NHGIS/census_1930/nhgis0059_ds67_1930_tract.csv',index_col=None, header=0)
census_1930_8 = census_1930_8.rename(columns={'BOJ001':'population',
                                          'BOK001':'white_native',
                                          'BOK002':'white_foreignparents',
                                          'BOK003':'white_foreign',
                                          'BOK004':'black',
                                        'BOK005':'other'})

census_1930_9 = pd.read_csv('data/NHGIS/census_1930/nhgis0059_ds68_1930_tract.csv',index_col=None, header=0)
census_1930_9 = census_1930_9.rename(columns={'BOO001':'population',
                                          'BPW001':'white_native',
                                          'BPW002':'white_foreignparents',
                                          'BPW003':'white_foreign',
                                          'BPW004':'black',
                                        'BPW005':'other'})

census_1930_10 = pd.read_csv('data/NHGIS/census_1930/nhgis0059_ds69_1930_tract.csv',index_col=None, header=0)
census_1930_10 = census_1930_10.rename(columns={'BQM001':'population'})

census_1930_11 = pd.read_csv('data/NHGIS/census_1930/nhgis0059_ds70_1930_tract.csv',index_col=None, header=0)
census_1930_11 = census_1930_11.rename(columns={'BQQ001':'population',
                                          'BRP001':'white_native',
                                          'BRP002':'white_foreignparents',
                                          'BRP003':'white_foreign',
                                          'BRP004':'black',
                                        'BRP005':'other'})

census_1930_12 = pd.read_csv('data/NHGIS/census_1930/nhgis0059_ds71_1930_tract.csv',index_col=None, header=0)
census_1930_12 = census_1930_12.rename(columns={'BRQ001':'population',
                                          'BRT001':'white_native',
                                          'BRT002':'white_foreignparents',
                                          'BRT003':'white_foreign',
                                          'BRT004':'black',
                                        'BRT005':'other'})
census_1930_13 = pd.read_csv('data/NHGIS/census_1930/nhgis0059_ds59_1930_tract.csv',index_col=None, header=0)
census_1930_13 = census_1930_13.rename(columns={'BI1001':'population',
                                          'BJR001':'white_native',
                                          'BJR002':'white_foreign',
                                          'BJR003':'black'}) ### no other

Aggregate all the cities

In [15]:
census_1930 = pd.concat([census_1930_1,census_1930_2,census_1930_3,census_1930_4,census_1930_5,
                       census_1930_6,census_1930_7,census_1930_8,census_1930_9,census_1930_10,
                       census_1930_11,census_1930_12,census_1930_13])
#### Cleanup
census_1930 = census_1930.drop_duplicates()
census_1930 = census_1930.fillna(0)

census_1930=census_1930[census_1930.population>0]
census_1930=census_1930.fillna(0)

Create percentages

In [16]:
census_1930['white']=census_1930['white_foreign']\
                        +census_1930['white_foreignparents']\
                        +census_1930['white_native']
        
census_1930['white_perc']=census_1930['white']/census_1930['population']
census_1930['black_perc']=census_1930['black']/census_1930['population']
census_1930['other_perc']=census_1930['other']/census_1930['population']


Upload to postgres

In [17]:
census_1930.to_sql('census_1930', engine,if_exists='replace')

### 1940
Using census from that year. Has the following columns: 
- Population
- Race by white, black, other
- College education

In [5]:
census_1940 = pd.read_csv('data/NHGIS/census_1940/nhgis0003_ds76_1940_tract.csv',index_col=None, header=0)
census_1940 = census_1940.rename(columns={'BUB001':'population',
                                          'BUQ001':'white',
                                          'BUQ002':'black',
                                          'BUH007':'college_male_1_3',
                                          'BUH016':'college_female_1_3',
                                          'BUH008':'college_male_4',
                                          'BUH017':'college_female_4',
                                         'GISJOIN':'gisjoin'})
census_1940['college'] = census_1940['college_male_1_3']+census_1940['college_female_1_3']\
                        +census_1940['college_male_4']+census_1940['college_female_4']
census_1940 = census_1940.drop_duplicates()
census_1940 = census_1940.fillna(0)

Create percentages

In [17]:
census_1940['white_perc']=census_1940['white']/census_1940['population']
census_1940['black_perc']=census_1940['black']/census_1940['population']

### No "other" category
census_1940['college_perc'] = (census_1940['college'])/ census_1940['population']

#### Cleanup
census_1940=census_1940[census_1940.population>0]
census_1940=census_1940.fillna(0)

Upload to postgres

In [18]:
census_1940.to_sql('census_1940', engine,if_exists='replace')

### 1950
Using census from that year. Has the following columns: 
- Population
- Race by white, black, other
- College education, 
- Median income
- Median value of home

In [138]:
census_1950 = pd.read_csv('data/NHGIS/census_1950/nhgis0021_ds82_1950_tract.csv',index_col=None, header=0)
census_1950 = census_1950.rename(columns={'BZ8001':'population',
                                          'B0J001':'white',
                                          'B0J002':'black',
                                          'B0J003':'other',
                                          'B0B008':'college_1_3',
                                          'B0B009':'college_4',
                                          'B0F001':'median_income',
                                         'GISJOIN':'gisjoin'})
census_1950['college'] = census_1950['college_1_3']+census_1950['college_4']

### Adjust for inflation from 1949
### from: http://www.in2013dollars.com/1949-dollars-in-2016
infl_factor_1950= 10.08433
census_1950['median_income_adj'] = infl_factor_1950*census_1950['median_income']
census_1950 = census_1950.drop_duplicates()
census_1950 = census_1950.fillna(0)

Create percentages

In [139]:
census_1950['white_perc']=census_1950['white']/(census_1950['white']+census_1950['black']+census_1950['other'])
census_1950['black_perc']=census_1950['black']/(census_1950['white']+census_1950['black']+census_1950['other'])
census_1950['other_perc']=census_1950['other']/(census_1950['white']+census_1950['black']+census_1950['other'])

census_1950['college_perc'] = (census_1950['college'] )/ census_1950['population']

### Adjust for inflation from 1949
### from: http://www.in2013dollars.com/1949-dollars-in-2016
infl_factor_1950= 10.08433
census_1950['median_income_adj']= infl_factor_1950*census_1950['median_income'].astype(float)

#### Cleanup
census_1950=census_1950[census_1950.population>0]
census_1950=census_1950[(census_1950['white']+census_1950['black']+census_1950['other'])>0]
census_1950=census_1950.fillna(0)

In [140]:
census_1950['median_income_adj'].mean()

28250.96270985663

Upload to postgres

In [21]:
census_1950.to_sql('census_1950', engine,if_exists='replace')

### 1960
Using census from that year. Has the following columns: 
- Population
- Race by white, black, other
- Education
- Income broken down by bins

For 1960, since there are only median income clases, we find the median income by simulating incomes based on income ranges and getting the median.

In [253]:
census_1960 = pd.read_csv('data/NHGIS/census_1960/nhgis0021_ds92_1960_tract.csv',index_col=None, header=0)
census_1960 = census_1960.rename(columns={'CA4001':'population',
                                          'B7B001':'white',
                                          'B7B002':'black',
                                          'B7B003':'other',
                                          'CA7001':'spanish_origin',
                                          'B8R007':'college_1_3',
                                          'B8R008':'college_4',
                                          'B8W001':'less_1000',
                                          'B8W002':'1000_1999',
                                          'B8W003':'2000_2999',
                                          'B8W004':'3000_3999',
                                          'B8W005':'4000_4999',
                                          'B8W006':'5000_5999',
                                          'B8W007':'6000_6999',
                                          'B8W008':'7000_7999',
                                          'B8W009':'8000_8999',
                                          'B8W010':'9000_9999',
                                          'B8W011':'10000_14999',
                                          'B8W012':'15000_24999',
                                          'B8W013':'25000_over',
                                         'GISJOIN':'gisjoin'})
census_1960['hispanic']=census_1960['spanish_origin']
census_1960['college']=(census_1960['college_1_3']+ census_1960['college_4'] )

### Since median income is measured by households instead of population, recreate a households category
census_1960['households']=(census_1960['less_1000']+census_1960['1000_1999']+census_1960['2000_2999']\
+census_1960['3000_3999']+census_1960['4000_4999'] +census_1960['5000_5999']\
+census_1960['6000_6999']+census_1960['7000_7999']+census_1960['8000_8999']\
+census_1960['9000_9999'] +census_1960['10000_14999']+census_1960['15000_24999']\
+census_1960['25000_over'])

census_1960 = census_1960.drop_duplicates()
census_1960 = census_1960.fillna(0)

### Adjust for inflation in 1959
### from: http://www.in2013dollars.com/1960-dollars-in-2016
infl_factor_1960=8.24766

census_1960['median_income']=census_1960.apply(lambda x:np.median(np.sort(np.concatenate([np.random.randint(0,999, size=int(x['less_1000'])),
        np.random.randint(1000,1999, size=int(x['1000_1999'])),
        np.random.randint(2000,2999, size=int(x['2000_2999'])),
        np.random.randint(3000,3999, size=int(x['3000_3999'])),
        np.random.randint(4000,4999, size=int(x['4000_4999'])),
        np.random.randint(5000,5999, size=int(x['5000_5999'])),
        np.random.randint(6000,6999, size=int(x['6000_6999'])),
        np.random.randint(7000,7999, size=int(x['7000_7999'])),
        np.random.randint(8000,8999, size=int(x['8000_8999'])),
        np.random.randint(9000,9999, size=int(x['9000_9999'])),
        np.random.randint(10000,14999, size=int(x['10000_14999'])),
        np.random.randint(15000,24999, size=int(x['15000_24999'])),
                                                             np.random.randint(25000,50000, size=int(x['25000_over']))
                                                            ]))),axis=1)


census_1960['median_income_adj']= infl_factor_1960*census_1960['median_income']

census_1960 = census_1960.drop_duplicates()
census_1960 = census_1960.fillna(0)

/Users/wenfeixu/anaconda2/envs/geo_wenfei/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/wenfeixu/anaconda2/envs/geo_wenfei/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [254]:
census_1960['median_income_adj'].median()

51312.81669

Create percentages

In [255]:
#### Population and race population don't add up
census_1960['white_perc']=census_1960['white']/(census_1960['white']+census_1960['black']+census_1960['other'])

census_1960['black_perc']=census_1960['black']/(census_1960['white']+census_1960['black']+census_1960['other'])
census_1960['hispanic_perc']=census_1960['spanish_origin']/(census_1960['population'])
census_1960['other_perc']=census_1960['other']/(census_1960['white']+census_1960['black']+census_1960['other'])

census_1960['college_perc'] = (census_1960['college'] )/ census_1960['population']

#### Cleanup
census_1960=census_1960[census_1960.population>0]
census_1960=census_1960[(census_1960['white']+census_1960['black']+census_1960['other'])>0]
census_1960=census_1960.fillna(0)

Upload to postgres

In [256]:
census_1960.to_sql('census_1960', engine,if_exists='replace')

### 1970
Using census from that year. Has the following columns: 
- Population
- Race by white, black, other
- Education
- Median Income
- Employment Status

For 1970, since the aggregate median incomes don't look correct. We use the median income clases and find the median income by simulating incomes based on income ranges and getting the median.

In [257]:
census_1970_1 = pd.read_csv('data/NHGIS/census_1970/nhgis0027_ds98_1970_tract.csv',index_col=None, header=0)
census_1970_2 = pd.read_csv('data/NHGIS/census_1970/nhgis0027_ds99_1970_tract.csv',index_col=None, header=0)
census_1970_3 = pd.read_csv('data/NHGIS/census_1970/nhgis0027_ds95_1970_tract.csv',index_col=None, header=0)
census_1970_4 = pd.read_csv('data/NHGIS/census_1970/nhgis0027_ds97_1970_tract.csv',index_col=None, header=0)
census_1970_5 = pd.read_csv('data/NHGIS/census_1970/nhgis0029_ds99_1970_tract.csv',index_col=None, header=0)




census_1970_1 = census_1970_1.rename(columns={'C1I001':'population',
                                          'C0X001':'white',
                                          'C0X002':'black',
                                          'C0X003':'other',
                                          'C06008':'college_1_3',
                                          'C06009':'college_4',
                                          'C06010':'college_5_more',
                                          'C07003':'unemployed',
                                         'GISJOIN':'gisjoin'})

census_1970_2 = census_1970_2.rename(columns={
                                          'C3T001':'less_1000',
                                          'C3T002':'1000_1999',
                                          'C3T003':'2000_2999',
                                          'C3T004':'3000_3999',
                                          'C3T005':'4000_4999',
                                          'C3T006':'5000_5999',
                                          'C3T007':'6000_6999',
                                          'C3T008':'7000_7999',
                                          'C3T009':'8000_8999',
                                          'C3T010':'9000_9999',
                                          'C3T011':'10000_11999',
                                          'C3T012':'12000_14999',
                                          'C3T013':'15000_24999',
                                          'C3T014':'25000_49999',
                                          'C3T015':'50000_over',
                                            'GISJOIN':'gisjoin'})



census_1970_2 = census_1970_2.fillna(0)
census_1970_2 = census_1970_2.replace(-1,0)

census_1970_2['median_income']=census_1970_2.apply(lambda x:np.median(np.sort(np.concatenate([np.random.randint(0,999, size=int(x['less_1000'])),
        np.random.randint(1000,1999, size=int(x['1000_1999'])),
        np.random.randint(2000,2999, size=int(x['2000_2999'])),
        np.random.randint(3000,3999, size=int(x['3000_3999'])),
        np.random.randint(4000,4999, size=int(x['4000_4999'])),
        np.random.randint(5000,5999, size=int(x['5000_5999'])),
        np.random.randint(6000,6999, size=int(x['6000_6999'])),
        np.random.randint(7000,7999, size=int(x['7000_7999'])),
        np.random.randint(8000,8999, size=int(x['8000_8999'])),
        np.random.randint(9000,9999, size=int(x['9000_9999'])),
        np.random.randint(10000,11999, size=int(x['10000_11999'])),
        np.random.randint(12000,14999, size=int(x['12000_14999'])),
        np.random.randint(15000,24999, size=int(x['15000_24999'])),
        np.random.randint(25000,49999, size=int(x['25000_49999'])),
        np.random.randint(50000,80000, size=int(x['50000_over']))
                                                            ]))),axis=1)


census_1970_3 = census_1970_3.rename(columns={'CEB003':'indian_m',
                                          'CEB004':'japanese_m',
                                          'CEB005':'chinese_m',
                                          'CEB006':'filipino_m',
                                          'CEB007':'hawaiian_m',
                                          'CEB008':'korean_m',
                                          'CEB012':'indian_f',
                                          'CEB013':'japanese_f',
                                          'CEB014':'chinese_f',
                                          'CEB015':'filipino_f',
                                          'CEB016':'hawaiian_f',
                                          'CEB017':'korean_f',
                                         'GISJOIN':'gisjoin'})

census_1970_3['asian']=census_1970_3['indian_m']+census_1970_3['japanese_m']+\
                                          census_1970_3['chinese_m']+\
                                          census_1970_3['filipino_m']+\
                                          census_1970_3['hawaiian_m']+\
                                          census_1970_3['korean_m']+\
                                          census_1970_3['indian_f']+\
                                          census_1970_3['japanese_f']+\
                                          census_1970_3['chinese_f']+\
                                          census_1970_3['filipino_f']+\
                                          census_1970_3['hawaiian_f']+\
                                          census_1970_3['korean_f']
census_1970_4 = census_1970_4.rename(columns={
                                          'CY9001':'spanish_origin',
                                            'GISJOIN':'gisjoin'})
census_1970_5 = census_1970_5.rename(columns={
                                          'C1K001':'median_income_agg',
                                            'GISJOIN':'gisjoin'})


census_1970 =census_1970_1.set_index('gisjoin')\
                        .join(census_1970_2[['gisjoin','median_income']].set_index('gisjoin'))\
                        .join(census_1970_3[['gisjoin','asian']].set_index('gisjoin'))\
                        .join(census_1970_4[['gisjoin','spanish_origin']].set_index('gisjoin'))\
                        .join(census_1970_5[['gisjoin','median_income_agg']].set_index('gisjoin')).reset_index()

            
census_1970['hispanic']=census_1970['spanish_origin']
census_1970['college']=census_1970['college_1_3']+census_1970['college_4']+census_1970['college_5_more']

### Adjust for inflation in 1969
### from: http://www.in2013dollars.com/1969-dollars-in-2016
infl_factor_1970=6.53970

census_1970['median_income_adj']= infl_factor_1970*census_1970['median_income']



#### Cleanup
census_1970 = census_1970.drop_duplicates()
census_1970 = census_1970.fillna(0)
census_1970=census_1970[census_1970.population>0]
census_1970=census_1970[(census_1970['white']+census_1970['black']+census_1970['other'])>0]


/Users/wenfeixu/anaconda2/envs/geo_wenfei/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/wenfeixu/anaconda2/envs/geo_wenfei/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/wenfeixu/anaconda2/envs/geo_wenfei/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [258]:
census_1970['median_income_adj'].median()

66005.1921

Create percentages

In [259]:
### Using the sume of races here becaucse I think population was a little wonky? 
census_1970['white_perc']=census_1970['white']/(census_1970['white']+census_1970['black']+census_1970['other'])
census_1970['asian_perc']=census_1970['asian']/(census_1970['white']+census_1970['black']+census_1970['other'])
                                              
census_1970['black_perc'] = census_1970['black']/(census_1970['white']+census_1970['black']+census_1970['other'])

census_1970['hispanic_perc']=census_1970['hispanic']/(census_1970['white']+census_1970['black']+census_1970['other'])
census_1970['other_perc']=census_1970['other']/(census_1970['white']+census_1970['black']+census_1970['other'])
census_1970['college_perc'] = (census_1970['college'] )/ (census_1970['white']+census_1970['black']+census_1970['other'])
census_1970['unemployed_perc'] = (census_1970['unemployed'] )/ (census_1970['white']+census_1970['black']+census_1970['other'])


Upload to postgres

In [260]:
census_1970.to_sql('census_1970', engine,if_exists='replace')

### 1980
Using census from that year. Has the following columns: 
- Population
- Race by white, black, other types of asian
- Education
- Median Income
- Employment Status

In [141]:
census_1980_1 = pd.read_csv('data/NHGIS/census_1980/nhgis0027_ds104_1980_tract.csv',index_col=None, header=0)
census_1980_2 = pd.read_csv('data/NHGIS/census_1980/nhgis0027_ds107_1980_tract.csv',index_col=None, header=0)



census_1980_1 = census_1980_1.rename(columns={'C7L001':'population',
                                          'C9D001':'white',
                                          'C9D002':'black',
                                          'C9D003':'american_indian',
                                          'C9D004':'eskimo',
                                          'C9D005':'aleut',
                                          'C9D006':'japanese',
                                          'C9D007':'chinese',
                                          'C9D008':'filipino',
                                          'C9D009':'korean',
                                          'C9D010':'asian_indian',
                                          'C9D011':'vietnamese',
                                          'C9D012':'hawaiian',
                                          'C9D013':'guamian',
                                          'C9D014':'samoan',
                                          'C9D015':'other',
                                          'C9E001':"non_spanish",
                                          'C9E002':'mexican',
                                           'C9E003':'puerto_rican',
                                              'C9E004':'cuban',
                                              'C9E005':'other_spanish',
                                          'GISJOIN':'gisjoin'})
census_1980_2 = census_1980_2.rename(columns={'DHM004':'college_1_3',
                                              'DHM005':'college_4',
                                              'DHX003':'unemployed_male',
                                              'DHX007':'unemployed_female',
                                              'DIE001':'median_income',
                                         'GISJOIN':'gisjoin'})

census_1980 =census_1980_1.set_index('gisjoin').join(census_1980_2[['gisjoin',
                                                                    'college_1_3',
                                                                    'college_4',
                                                                    'unemployed_male',
                                                                    'unemployed_female',
                                                                    'median_income']].set_index('gisjoin')).reset_index()
census_1980['other']=(census_1980['american_indian']+census_1980['eskimo']+census_1980['aleut']+census_1980['japanese']+\
                    census_1980['chinese']+census_1980['filipino']+census_1980['korean']+census_1980['asian_indian']+\
                    census_1980['vietnamese']+census_1980['hawaiian']+census_1980['guamian']+census_1980['samoan']+census_1980['other'])

census_1980['hispanic']= (census_1980['mexican']+census_1980['puerto_rican']+census_1980['cuban']+census_1980['other_spanish'])
census_1980['college'] =census_1980['college_1_3']+census_1980['college_4']
census_1980['unemployed'] =census_1980['unemployed_male']+census_1980['unemployed_female']

### Adjust for inflation in 1979
### from: http://www.in2013dollars.com/1950-dollars-in-2016
infl_factor_1980=3.30588
census_1980['median_income_adj']= infl_factor_1980*(census_1980['median_income']).astype(float)

#### Cleanup
census_1980=census_1980[census_1980.population>0]
census_1980 = census_1980.drop_duplicates()
census_1980 = census_1980.fillna(0)

Create percentages

In [142]:
census_1980['white_perc']=census_1980['white']/census_1980['population']
census_1980['black_perc']=census_1980['black']/census_1980['population']
census_1980['hispanic_perc'] = (census_1980['hispanic'])/(census_1980['population'])
census_1980['other_perc']=(census_1980['other']) /census_1980['population']
census_1980['college_perc'] = (census_1980['college'] )/ census_1980['population']
census_1980['unemployed_perc'] = (census_1980['unemployed'] )/ census_1980['population']


Upload to postgres

In [60]:
census_1980.to_sql('census_1980', engine,if_exists='replace')

## Starting 1990 we use block groups

### 1990

Using census from that year. Has the following columns: 
- Population
- Race by white, black, other
- Education
- Median Income
- Employment Status

In [7]:
census_1990_1 = pd.read_csv('/data/NHGIS/census_1990/nhgis0023_ds120_1990_blck_grp.csv',index_col=None, header=0)
census_1990_2 = pd.read_csv('/data/NHGIS/census_1990/nhgis0023_ds123_1990_blck_grp.csv',index_col=None, header=0)
census_1990_1['PMSAA']=census_1990_1['PMSAA'].astype(str)
census_1990_1['MSA_CMSAA']=census_1990_1['MSA_CMSAA'].astype(str)
census_1990_2['PMSAA']=census_1990_2['PMSAA'].astype(str)
census_1990_2['MSA_CMSAA']=census_1990_2['MSA_CMSAA'].astype(str)

census_1990_1 = census_1990_1.rename(columns={'ET1001':'population',
                                          'ET2001':'white',
                                          'ET2002':'black',
                                          'ET2003':'native_american',
                                          'ET2004':'asian',
                                          'ET2005':'other',
                                          'ET2006':'white_hispanic',
                                          'ET2007':'black_hispanic',
                                          'ET2008':'indian_hispanic',
                                          'ET2009':'asian_hispanic',
                                          'ET2010':'other_hispanic',
                                          'EST001':'median_value',
                                          'GISJOIN':'gisjoin'})
census_1990_1['hispanic']=census_1990_1['white_hispanic']+census_1990_1['black_hispanic']\
                            +census_1990_1['indian_hispanic']+census_1990_1['asian_hispanic']+census_1990_1['other_hispanic']
census_1990_1['other']=census_1990_1['other']+census_1990_1['asian']+census_1990_1['native_american']
census_1990_2 = census_1990_2.rename(columns={'E33004':'college_notcomplete',
                                              'E33005':'college_1_3',
                                              'E33006':'college_4',
                                              'E33007':'graduate',
                                              'E4I003':'unemployed_male',
                                              'E4I007':'unemployed_female',
                                              'E4U001':'median_income',
                                         'GISJOIN':'gisjoin'})
census_1990_2['college']=census_1990_2['college_notcomplete']+census_1990_2['college_1_3']\
                            + census_1990_2['college_4']+census_1990_2['graduate']
census_1990_2['unemployed']=census_1990_2['unemployed_female']+census_1990_2['unemployed_male']

### Adjust for inflation in 1989
### from: http://www.in2013dollars.com/1950-dollars-in-2016
infl_factor_1990=1.93554
census_1990_2['median_income_adj']= infl_factor_1990*(census_1990_2['median_income']).astype(float)

census_1990 =census_1990_1.set_index('gisjoin').join(census_1990_2[['gisjoin',
                                                                    "college",
                                                                    'unemployed',
                                                                    'median_income_adj',
                                                                    'median_income']].set_index('gisjoin')).reset_index()

#### Cleanup 
census_1990=census_1990[census_1990.population>0]
census_1990 = census_1990.drop_duplicates()
census_1990 = census_1990.fillna(0)

In [ ]:
# census_1990[census_1990['unemployed_perc']>1][['unemployed','population']]
# [['college_perc','college','population']].sort_values('college_perc',ascending=False)

Create percentages

In [ ]:
census_1990['white_perc']=census_1990['white']/census_1990['population']
census_1990['black_perc']=census_1990['black']/census_1990['population']
census_1990['other_perc']=(census_1990['other']) /census_1990['population']
census_1990['hispanic_perc']=(census_1990['hispanic']) /census_1990['population']

census_1990['college_perc'] = (census_1990['college'])/ census_1990['population']
census_1990['unemployed_perc']= (census_1990['unemployed'])/census_1990['population'].astype(float)



Upload to postgres

In [5]:
census_1990.to_sql('census_1990', engine,if_exists='replace')

### 2000
Using census from that year. Has the following columns: 
- Population
- Race by white, black, other
- Education
- Median Income
- Employment Status

In [6]:
census_2000_1 = pd.read_csv('data/NHGIS/census_2000/nhgis0023_ds147_2000_blck_grp.csv',index_col=None, header=0)
census_2000_2 = pd.read_csv('data/NHGIS/census_2000/nhgis0023_ds152_2000_blck_grp.csv',index_col=None, header=0)


census_2000_1 = census_2000_1.rename(columns={'FXS001':'population',
                                          'FYF001':'white',
                                          'FYF002':'black',
                                          'FYF003':'native_american',
                                          'FYF004':'asian',
                                          'FYF005':'pacific_islander',
                                          'FYF008':'white_hispanic',
                                          'FYF009':'black_hispanic',
                                          'FYF010':'indian_hispanic',
                                          'FYF011':'asian_hispanic',
                                          'FYF012':'islander_hispanic',
                                          'FYF013':'other_hispanic',
                                          'FYF014':'twoother_hispanic',
                                          'FYE006':'one_other',
                                          'FYE007':'two_other',
                                          'GISJOIN':'gisjoin'})

census_2000_1['hispanic']=census_2000_1['white_hispanic']+census_2000_1['black_hispanic']+\
                        census_2000_1['indian_hispanic']+census_2000_1['asian_hispanic']+\
                        census_2000_1['islander_hispanic']+census_2000_1['other_hispanic']+census_2000_1['twoother_hispanic']

census_2000_1['other']=census_2000_1['native_american']+census_2000_1['asian']+census_2000_1['pacific_islander']+census_2000_1['one_other']+census_2000_1['two_other']


census_2000_2 = census_2000_2.rename(columns={'HD1011':'somecollege_male',
                                              'HD1012':'college_1_3_male',
                                              'HD1013':'college_4_male',
                                              'HD1014':'masters_male',
                                              'HD1015':'professional_male',
                                              'HD1016':'doctorate_male',
                                              'HD1027':'somecollege_female',
                                              'HD1028':'college_1_3_female',
                                              'HD1029':'college_4_female',
                                              'HD1030':'masters_female',
                                              'HD1031':'professional_female',
                                              'HD1032':'doctorate_female',
                                              'HEZ002':'unemployed_male',
                                              'HEZ004':'unemployed_female',
                                              'HF6001':'median_income',
                                         'GISJOIN':'gisjoin'})
census_2000_2['college'] = census_2000_2['somecollege_male']+ census_2000_2['college_1_3_male']+census_2000_2['college_4_male']\
                            +census_2000_2['masters_male']+census_2000_2['professional_male']+census_2000_2['doctorate_male']\
                            +census_2000_2['somecollege_female']+ census_2000_2['college_1_3_female']+census_2000_2['college_4_female']\
                            +census_2000_2['masters_female']+census_2000_2['professional_female']+census_2000_2['doctorate_female']\

census_2000_2['unemployed']=census_2000_2['unemployed_male']+census_2000_2['unemployed_female']
census_2000 =census_2000_1.set_index('gisjoin').join(census_2000_2[['gisjoin',
                                             'college',
                                              'unemployed',
                                             'median_income']].set_index('gisjoin')).reset_index()

### Adjust for inflation in 1999
### from: http://www.in2013dollars.com/1950-dollars-in-2016
infl_factor_2000=1.44062
census_2000['median_income_adj']= infl_factor_2000*(census_2000['median_income']).astype(float)

### Cleanup
census_2000=census_2000[census_2000.population>0]
census_2000 = census_2000.drop_duplicates()
census_2000= census_2000.fillna(0)

In [7]:
census_2000['median_income'].mean()

44531.14644084084

Create percentages

In [8]:
census_2000['white_perc']=census_2000['white']/census_2000['population']
census_2000['black_perc']=census_2000['black']/census_2000['population']
census_2000['other_perc']=(census_2000['other']) /census_2000['population']
census_2000['hispanic_perc']=(census_2000['hispanic']) /census_2000['population']

census_2000['college_perc'] = (census_2000['college'] )/ census_2000['population']
census_2000['unemployed_perc']= (census_2000['unemployed'])/census_2000['population'].astype(float)



Upload to Postgres

In [9]:
census_2000.to_sql('census_2000', engine,if_exists='replace')

### 2010
**Starting 2010, longform census no longer exists - replaced with ACS data. **

Using census from that year and 2008-2012 ACS. Has the following columns: 
- Population
- Race by white, black, other
- Education
- Median Income
- Employment Status

In [57]:
### Had to change encoding to 'latin-1'
census_2010_1 = pd.read_csv('data/NHGIS/census_2010/nhgis0023_ds171_2010_blck_grp.csv',index_col=None, header=0,encoding='latin-1')
census_2010_2 = pd.read_csv('data/NHGIS/census_2010/nhgis0023_ds172_2010_blck_grp.csv',index_col=None, header=0,encoding='latin-1')
census_2010_3 = pd.read_csv('data/NHGIS/census_2010/nhgis0023_ds191_20125_2012_blck_grp.csv',index_col=None, header=0,encoding='latin-1')

census_2010_1 = census_2010_1.rename(columns={
                                            'H7R001':'population',
                                            'H7R002':'hispanic',
                                          'H7R005':'white',
                                          'H7R006':'black',
                                          'H7R007':'native_american',
                                          'H7R008':'asian',
                                          'H7R009':'pacific_islander',
                                          'H7R010':'one_other',
                                          'H7R011':'two_other',
                                            'H7R028':'three_other',
                                          'H7R049':'four_other',
                                            'H7R065':'five_other',
                                            'H7R072':'six_other',
                                          'GISJOIN':'gisjoin'})
census_2010_1['other']=census_2010_1['native_american']+census_2010_1['asian']+\
                        census_2010_1['pacific_islander']+census_2010_1['one_other']+\
                        census_2010_1['two_other']+census_2010_1['three_other']+census_2010_1['four_other']+\
                        census_2010_1['five_other']+census_2010_1['six_other']


census_2010_3 = census_2010_3.rename(columns={'QUSE020':'somecollege',
                                              'QUSE021':'college_1_3',
                                              'QUSE022':'college_4',
                                              'QUSE023':'masters',
                                              'QUSE024':'professional',
                                              'QUSE025':'doctorate',
                                              'QU1E001':'median_income',
                                              'QXSE005':'unemployed',
                                              'QUSE001':'edu_total',
                                         'GISJOIN':'gisjoin'})
census_2010_3['college'] =census_2010_3['somecollege'] +census_2010_3['college_1_3'] +census_2010_3['college_4']\
                        +census_2010_3['masters'] +census_2010_3['professional'] +census_2010_3['doctorate']
census_2010 =census_2010_1[['population','white','black','hispanic','other','gisjoin']].set_index('gisjoin').join(census_2010_3[['gisjoin',
                                            'college','edu_total',
                                            'unemployed',
                                             'median_income']].set_index('gisjoin')).reset_index()



### Adjust for inflation in 2012, the values are in 2012 inflation adjusted dollars
infl_factor_2012=1.04535
census_2010['median_income_adj']= infl_factor_2012*(census_2010['median_income']).astype(float)

### Cleanup
census_2010=census_2010[census_2010.population>0]
census_2010=census_2010.fillna(0)

census_2010 = census_2010.drop_duplicates()
census_2010 = census_2010.fillna(0)

Create percentages

In [70]:
census_2010['white_perc']=census_2010['white']/census_2010['population']
census_2010['black_perc']=census_2010['black']/census_2010['population']
census_2010['other_perc']=(census_2010['other']) /census_2010['population']
census_2010['hispanic_perc']=(census_2010['hispanic']) /census_2010['population']
census_2010['college_perc'] = (census_2010['college'])/ census_2010['edu_total']
census_2010['unemployed_perc']= (census_2010['unemployed'])/census_2010['population'].astype(float)


Upload to postgres

In [71]:
census_2010.to_sql('census_2010', engine,if_exists='replace')

### 2016
**2016 1yr ACS data. **

Has the following columns: 
- Population
- Race by white, black, other
- Education
- Median Income
- Employment Status

In [37]:
### Had to change encoding to 'latin-1'
census_2016 = pd.read_csv('data/NHGIS/census_2016/nhgis0024_ds225_20165_2016_blck_grp.csv',index_col=None, header=0,encoding='latin-1')

census_2016 = census_2016.rename(columns={'AF2LE001':'population',
                                          'AF2UE003':'white',
                                          'AF2UE004':'black',
                                          'AF2UE005':'native_american',
                                          'AF2UE006':'asian',
                                          'AF2UE007':'pacific_islander',
                                          'AF2ME007':'one_other',
                                          'AF2ME008':'two_other',
                                          'AF2UE013':'hispanic_white',
                                          'AF2UE014':'hispanic_black',
                                          'AF2UE015':'hispanic_indian',
                                          'AF2UE016':'hispanic_asian',
                                          'AF2UE017':'hispanic_islander',
                                          'AF2UE018':'hispanic_other',
                                          'AF2UE019':'hispanic_twoother',
                                          'AF2UE020':'hispanic_twootherother',
                                          'AF4OE020':'somecollege',
                                              'AF4OE021':'college_1_3',
                                              'AF4OE022':'college_4',
                                              'AF4OE023':'masters',
                                              'AF4OE024':'professional',
                                              'AF4OE025':'doctorate',
                                              'AF49E001':'median_income',
                                              'AF67E005':'unemployed',
                                              'AF9LE001':'median_value',
                                         'GISJOIN':'gisjoin'})
census_2016['hispanic']=census_2016['hispanic_white']+census_2016['hispanic_black']+census_2016['hispanic_indian']+census_2016['hispanic_asian']\
                        +census_2016['hispanic_islander']+census_2016['hispanic_other']+census_2016['hispanic_twoother']+census_2016['hispanic_twootherother']
census_2016['other']=census_2016['native_american']+census_2016['asian']+census_2016['pacific_islander']+census_2016['one_other']+census_2016['two_other']
census_2016['college']= census_2016['somecollege']+census_2016['college_1_3']+census_2016['college_4']+census_2016['masters']\
                        + census_2016['professional']+census_2016['doctorate']

census_2016['median_income_adj']= (census_2016['median_income']).astype(float)

### Cleanup
census_2016=census_2016[census_2016.population>0]
census_2016 = census_2016.drop_duplicates()
census_2016 = census_2016.fillna(0)
cols_keep = ['gisjoin','population','white','black','other','native_american','asian','pacific_islander','hispanic','college','median_income_adj','unemployed']
census_2016 = census_2016[cols_keep]

Create percentages

In [38]:
census_2016['white_perc']=census_2016['white']/census_2016['population']
census_2016['black_perc']=census_2016['black']/census_2016['population']
census_2016['other_perc']=(census_2016['other']) /census_2016['population']
census_2016['hispanic_perc']=(census_2016['hispanic']) /census_2016['population']
census_2016['college_perc'] = (census_2016['college'])/ census_2016['population']
census_2016['unemployed_perc']= (census_2016['unemployed'])/census_2016['population'].astype(float)



Upload to postgres

In [39]:
census_2016.to_sql('census_2016', engine,if_exists='replace')

*****

## There are some duplicates in the census data
They look like this: 
```
152	Tract   K0001     in Boston MA	4258	**1389**	**1414**	1200	C	Boston	4806	4003	0.9401127289807421	0.0577736026303429	0.00211366838891498	C	0.233629061563406	0.234096988926357

233	Tract   K0001     in Boston MA	4258	**1388**	**1413**	1200	C	Boston	4806	4001	0.9396430248943169	0.0577736026303429	0.00211366838891498	C	0.233629061563406	0.234096988926357
```
We'll run for example:
```
select distinct on ("AREANAME")
* from census_1930;
```

We'll remove these in the weighted area joins